In [22]:
import numpy as np
from collections import OrderedDict

### 합성곱/풀링 계층 구현하기

In [23]:
x = np.random.rand(10,1,28,28)

In [24]:
x.shape

(10, 1, 28, 28)

In [25]:
x[0].shape

(1, 28, 28)

In [26]:
x[9][0]   # 10번째 데이터, 첫 번째 채널

array([[1.64563060e-01, 6.38914675e-01, 4.48366789e-01, 6.43278703e-01,
        8.38943523e-01, 6.75190961e-01, 2.03637694e-01, 7.05712644e-01,
        9.80640741e-01, 5.25520926e-01, 1.36988470e-01, 8.82150587e-01,
        6.58909685e-01, 1.51637017e-01, 5.00535380e-01, 3.64410809e-01,
        4.25582866e-01, 1.73551645e-01, 3.79566162e-01, 6.82111337e-01,
        9.60437949e-01, 7.29815487e-01, 2.40567476e-01, 9.55796856e-01,
        2.43011465e-02, 5.02613174e-02, 6.92665202e-02, 6.67868782e-01],
       [8.36908972e-01, 7.25223412e-01, 1.77967378e-03, 6.62608635e-02,
        1.03038298e-01, 4.16238714e-01, 1.59236489e-01, 1.14308901e-01,
        3.67900663e-01, 4.70002505e-01, 6.08247386e-01, 4.87461961e-01,
        2.02472863e-01, 5.76514774e-04, 3.46992395e-01, 9.18584179e-01,
        1.29886459e-01, 4.94881256e-01, 2.89878943e-01, 5.35537882e-02,
        8.23536107e-01, 6.90603424e-01, 9.99601429e-01, 2.64861539e-01,
        1.30422972e-01, 4.68064445e-02, 9.33338039e-01, 5.82204

### im2col로 데이터 전개하기, 입력 데이터의 차원을 행렬로 바꿔준다.  

In [27]:
# im2col은 입력 데이터(특징 맵)를 필터링(가중치 계산)하기 좋게 전개하는(펼치는) 함수이다. 
# 3차원 입력 데이터에 im2col을 적용하면 2차원 행렬로 바뀐다. 배치 안의 데이터 수까지 포함하여 4차원 데이터를 2차원으로 변환한다.

def im2col(input_data, filter_h, filter_w, stride=1, pad=0): # input_data: (데이터 수, 채널 수, 높이, 너비)의 4차원 배열로 이루어진 입력 데이터, filter_h: 필터의 높이, filter_w: 필터의 너비, stride: 스트라이드, pad: 패딩
    N, C, H, W = input_data.shape               # N: 데이터 수, C: 채널 수, H: 높이, W: 너비
    out_h = (H + 2*pad - filter_h)//stride + 1  # 출력 데이터의 높이 OH, fiter_h는 필터의 높이
    out_w = (W + 2*pad - filter_w)//stride + 1  # 출력 데이터의 너비 OW, filter_w는 필터의 너비

    img = np.pad(input_data, [(0,0), (0,0), (pad, pad), (pad, pad)], 'constant') # 입력 데이터에 pad만큼 0으로 채움
    col = np.zeros((N, C, filter_h, filter_w, out_h, out_w)) # 입력 데이터를 2차원 행렬로 전개할 배열 0으로 초기화
    print("col.shape: ", col.shape) # (데이터 수, 채널 수, 필터의 높이, 필터의 너비, 출력 데이터의 높이, 출력 데이터의 너비)
    print(col)

    # 입력 이미지 데이터를 슬라이딩 윈도우 방식으로 필터링하여 2차원 배열로 변환
    # 필터의 높이, 너비만큼 이동하면서 필터를 적용
    for y in range(filter_h):
        y_max = y + stride*out_h
        for x in range(filter_w):
            x_max = x + stride*out_w
            col[:, :, y, x, :, :] = img[:, :, y:y_max:stride, x:x_max:stride]

    col = col.transpose(0, 4, 5, 1, 2, 3).reshape(N*out_h*out_w, -1)
    return col # 2차원 행렬로 전개


In [28]:
x1 = np.random.rand(1, 2, 3, 3) # (데이터 수, 채널 수, 높이, 너비), input_data
col1 = im2col(x1, 2, 2, stride=1, pad=0) # 필터 크기 5x5, 스트라이드 1, 패딩 0
print(col1)

col.shape:  (1, 2, 2, 2, 2, 2)
[[[[[[0. 0.]
     [0. 0.]]

    [[0. 0.]
     [0. 0.]]]


   [[[0. 0.]
     [0. 0.]]

    [[0. 0.]
     [0. 0.]]]]



  [[[[0. 0.]
     [0. 0.]]

    [[0. 0.]
     [0. 0.]]]


   [[[0. 0.]
     [0. 0.]]

    [[0. 0.]
     [0. 0.]]]]]]
[[0.01966897 0.25289289 0.87514524 0.67231562 0.59706941 0.28248729
  0.23195223 0.75457786]
 [0.25289289 0.25706515 0.67231562 0.41921015 0.28248729 0.68526451
  0.75457786 0.84046289]
 [0.87514524 0.67231562 0.98100203 0.48524343 0.23195223 0.75457786
  0.10188267 0.24994754]
 [0.67231562 0.41921015 0.48524343 0.17686588 0.75457786 0.84046289
  0.24994754 0.38520659]]


### 합성곱 계층 구현  

In [8]:
class Convolution:
  def __init__(self,W,b,stride=1,pad=0):
    self.W = W            # 필터(가중치)
    self.b = b            # 편향
    self.stride = stride  # 스트라이드
    self.pad = pad        # 패딩
  
  def forward(self,x):
    FN, C, FH, FW = self.W.shape  # 필터, FN: 필터 개수, C: 채널 수, FH: 필터 높이, FW: 필터 너비
    N, C, H, W = x.shape          # 입력 데이터, N: 데이터 수, C: 채널 수, H: 높이, W: 너비
    out_h = 1 + int((H + 2*self.pad - FH) / self.stride) # 출력 데이터 높이, OH
    out_w = 1 + int((W + 2*self.pad - FW) / self.stride) # 출력 데이터 너비, OW

    col = im2col(x, FH, FW, self.stride, self.pad) # 입력 데이터를 2차원 배열로 전개한 col
    col_W = self.W.reshape(FN, -1).T               # 필터를 reshape를 사용하여 2차원 배열로 전개한 col_W, T는 transpose를 의미

    out = np.dot(col, col_W) + self.b              # 합성곱 연산 수행 (행렬의 내적)
    out = out.reshape(N, out_h, out_w, -1).transpose(0, 3, 1, 2)  # 출력 데이터를 reshape하여 transpose, 형상을 (N(0), H(1), W(2), C(3))에서 (N(0), C(3), H(1), W(2))로 변경

    return out

In [9]:
x1 = np.random.rand(1, 3, 7, 7) # (데이터 수, 채널 수, 높이, 너비), input_data
col1 = im2col(x1, 5, 5, stride=1, pad=0) # 필터 크기 5x5, 스트라이드 1, 패딩 0
print(col1)                              # input_data를 2차원 배열로 전개

# Convolution 클래스 사용
W = np.random.rand(3, 3, 5, 5) # (필터 수, 채널 수, 높이, 너비), filter 5x5x3
b = np.random.rand(1)          # 편향
conv1 = Convolution(W, b, stride=1, pad=0) # Convolution 클래스 생성

out1 = conv1.forward(x1) # 순전파
print(out1)              # iput_data에 필터를 적용한 결과 출력

[[0.09660822 0.19535746 0.29024376 0.40440087 0.87725305 0.3677997
  0.36744016 0.71314939 0.66949363 0.61916521 0.00904648 0.44031811
  0.24534473 0.63486327 0.3409896  0.14953188 0.77823654 0.09859328
  0.45635829 0.29527894 0.94754112 0.32872223 0.20713547 0.98577513
  0.15904226 0.14410558 0.95231034 0.19201407 0.89835769 0.5234781
  0.11548592 0.99094856 0.77733692 0.54720339 0.3026326  0.43042961
  0.2451785  0.39980013 0.11008696 0.21738337 0.60974619 0.07457432
  0.94413584 0.19782117 0.05535637 0.14965147 0.32405318 0.48830658
  0.8853836  0.26217093 0.34909821 0.15969376 0.52127263 0.6539688
  0.70162616 0.9477525  0.76355933 0.53432832 0.42002965 0.72184873
  0.22179259 0.52177492 0.44609345 0.09766681 0.20875351 0.45819082
  0.65714753 0.05525377 0.84797755 0.27340176 0.25885672 0.2885672
  0.7458426  0.53221541 0.7335434 ]
 [0.19535746 0.29024376 0.40440087 0.87725305 0.11096718 0.36744016
  0.71314939 0.66949363 0.61916521 0.79523875 0.44031811 0.24534473
  0.63486327 0.3

### Pooling 계층  

In [10]:
class Pooling:
  def __init__(self,pool_h,pool_w,stride=1,pad=0):
    self.pool_h = pool_h  # 풀링 높이
    self.pool_w = pool_w  # 풀링 너비
    self.stride = stride  # 스트라이드
    self.pad = pad        # 패딩
    
  def forward(self,x):
    N, C, H, W = x.shape
    print(x.shape)
    out_h = int(1 + (H - self.pool_h) / self.stride) # 출력 데이터 높이 OH
    out_w = int(1 + (W - self.pool_w) / self.stride) # 출력 데이터 너비 OW
    
    # 전개(1)
    col = im2col(x, self.pool_h, self.pool_w, self.stride, self.pad)
    print(col)
    col = col.reshape(-1, self.pool_h*self.pool_w)
    print(col)
    
    
    # 최댓값(2)
    out = np.max(col, axis=1) # Max pooling, 각 행마다 최댓값을 구함
    
    # 성형(3)
    out = out.reshape(N, out_h, out_w, C).transpose(0, 3, 1, 2)
    
    return out

In [11]:
# out1(합성곱 층 결과) 을 Pooling 클래스의 forward 메서드에 넣어 풀링 연산을 수행

pool1 = Pooling(pool_h=2, pool_w=2, stride=1) # Pooling 클래스 생성, 풀링 높이 2, 풀링 너비 2, 스트라이드 1
out2 = pool1.forward(out1) # Pooling 순전파
print(out2)                # out1에 풀링을 적용한 결과 출력

(1, 3, 3, 3)
[[18.50979086 19.93175626 19.075306   19.87312806 19.46815909 19.92374594
  20.52634064 20.72230662 17.9498556  18.81280244 16.30257186 18.20549764]
 [19.93175626 21.07567471 19.87312806 18.67900209 19.92374594 20.92311387
  20.72230662 19.72608798 18.81280244 18.35201445 18.20549764 18.52876881]
 [19.075306   19.87312806 16.97023393 18.13983386 20.52634064 20.72230662
  18.35097549 18.47509569 16.30257186 18.20549764 15.42220372 17.8993118 ]
 [19.87312806 18.67900209 18.13983386 18.64131186 20.72230662 19.72608798
  18.47509569 19.22479772 18.20549764 18.52876881 17.8993118  18.47559988]]
[[18.50979086 19.93175626 19.075306   19.87312806]
 [19.46815909 19.92374594 20.52634064 20.72230662]
 [17.9498556  18.81280244 16.30257186 18.20549764]
 [19.93175626 21.07567471 19.87312806 18.67900209]
 [19.92374594 20.92311387 20.72230662 19.72608798]
 [18.81280244 18.35201445 18.20549764 18.52876881]
 [19.075306   19.87312806 16.97023393 18.13983386]
 [20.52634064 20.72230662 18.3509

### CNN 구현하기   

### 필요한 계층 불러오기  

In [12]:
# 순전파와 역전파를 포함한 ReLU 계층 구현

class Relu: 
  def __init__(self):
    self.mask = None # mask는 True/False로 구성된 넘파이 배열, 순전파의 입력인 x의 원소 값이 0 이하인 인덱스는 True, 그 외(0보다 큰 원소)는 False로 유지
  
  # 순전파
  def forward(self,x):
    self.mask = (x <= 0) # x의 원소 값이 0 이하인 인덱스를 True로 설정
    out = x.copy() # 입력 x를 복사하여 out에 저장
    out[self.mask] = 0 # mask의 원소가 True인 인덱스에 대응하는 원소를 0으로 설정, 즉 x가 0보다 작으면 0으로 변환, 그 외는 그대로 유지
    
    return out
  
  # 역전파
  def backward(self,dout):
    dout[self.mask] = 0
    dx = dout # 순전파 때의 입력인 x가 0보다 작으면 역전파 때의 값은 0, 그 외는 상류 값을 그대로 전달한다. 
    
    return dx

In [13]:
# 신경망의 순전파 때 수행하는 행렬의 내적을 기하학에서는 어파인 변환(Affine Transformation)이라고 불러서 Affine 계층이라는 이름을 사용한다.
# Affine 계층은 가중치 신호의 총합을 계산하고, 편향을 더한다.

# N개의 데이터를 묶어 순전파와 역전파를 수행한 Affine 계층 구현
class Affine: 
  def __init__(self,w,b):
    self.w = w  # 가중치
    self.b = b  # 편향
    self.x = None # 입력
    self.dw = None # 가중치의 미분
    self.db = None # 편향의 미분
   
  # 순전파  
  def forward(self, x):
    self.x = x
    out = np.dot(x,self.w) + self.b  # y = WX + B
    
    return out
   
  # 역전파  
  def backward(self,dout):
    dx = np.dot(dout,self.w.T)  # dx = dL/dy * W^T, self.w.T는 self.w의 전치행렬, 입력 x에 대한 상류 값의 미분으로 이전 층으로 전파해야 할 값이다
    self.dw = np.dot(self.x.T,dout) # dw = X^T * dL/dy, self.x.T는 self.x의 전치행렬
    self.db = np.sum(dout, axis = 0) # db = dL/dy의 각 원소의 총합이다, axis = 0은 행 방향으로 더한다
    
    return dx

In [14]:
def softmax(x):
    if x.ndim == 2:
        x = x - np.max(x, axis=1, keepdims=True) # 오버플로 대책
        x = np.exp(x)
        x /= np.sum(x, axis=1, keepdims=True)
    elif x.ndim == 1:
        x = x - np.max(x) # 오버플로 대책
        x = np.exp(x) / np.sum(np.exp(x))
    return x
  
  # 평균 손실함수
def cross_entropy_error(y, t):
    if y.ndim == 1:
        t = t.reshape(1, t.size) # t는 실제 정답 레이블
        y = y.reshape(1, y.size) # y는 신경망의 출력

    batch_size = y.shape[0]
    return -np.sum(t * np.log(y + 1e-7)) / batch_size  # np.log(0)이 -inf가 되는 것을 방지하기 위해 아주 작은 delta(1e-7)를 더해줌

# Softmax-with-Loss 계층

class SoftmaxWithLoss:
  def __init__(self):
    self.loss = None # 손실
    self.y = None # softmax의 출력
    self.t = None # 정답 레이블(원-핫 인코딩 형태)
    
  def forward(self,x,t):
    self.t = t
    self.y = softmax(x)
    self.loss = cross_entropy_error(self.y,self.t)
    
    # 순전파의 결과인 손실을 반환한다
    return self.loss
  
  def backward(self,dout=1):
    batch_size = self.t.shape[0]
    dx = (self.y - self.t) / batch_size # 역전파의 결과로는 데이터의 개수로 나눠서 데이터 1개당 오차를 앞 계층으로 전파한다
    
    return dx

In [15]:
# {Convolution-ReLU-Pooling}-{Affine-ReLU}-{Affine-Softmax} 구현

class SimpleConvNet:
  def __init__(self,input_dim=(1,28,28),                                        # 입력 데이터(채널 수, 높이, 너비) 1x28x28
               conv_param={'filter_num':30,'filter_size':5,'pad':0,'stride':1}, # 합성곱 계층의 하이퍼파라미터(필터 개수(FN), 필터 크기(FH,FW), 패딩, 스트라이드)
               hidden_size=100,output_size=10,weight_init_std=0.01):            # 은닉층 뉴런 수, 출력층의 뉴런 수(클래스 수), 가중치 초기화 표준편차
    filter_num = conv_param['filter_num']   # 필터 개수, FN
    filter_size = conv_param['filter_size'] # 필터 크기, FH, FW
    filter_pad = conv_param['pad']          # 패딩
    filter_stride = conv_param['stride']    # 스트라이드
    input_size = input_dim[1]               # 입력 데이터(채널 수, 높이, 너비)의 높이
    conv_output_size = (input_size - filter_size + 2*filter_pad) / filter_stride + 1 # 합성곱 계층 출력 크기 30x24x24
    pool_output_size = int(filter_num * (conv_output_size/2) * (conv_output_size/2)) # 풀링 계층 출력 크기 30x12x12
    
    # 1층의 합성곱 계층과 2, 3층의 완전연결 계층의 가중치와 편향 생성
    self.params = {}
    # 합성곱 계층의 가중치와 편향
    self.params['W1'] = weight_init_std * np.random.randn(filter_num, input_dim[0], filter_size, filter_size)
    self.params['b1'] = np.zeros(filter_num)
    
    # 2층 완전연결 계층의 가중치와 편향
    self.params['W2'] = weight_init_std * np.random.randn(pool_output_size, hidden_size)
    self.params['b2'] = np.zeros(hidden_size)
    
    # 3층 완전연결 계층의 가중치와 편향
    self.params['W3'] = weight_init_std * np.random.randn(hidden_size, output_size)
    self.params['b3'] = np.zeros(output_size)
    
    # CNN 구성 계층 생성
    self.layers = OrderedDict()
    # 1층 합성곱 계층
    self.layers['Conv1'] = Convolution(self.params['W1'], self.params['b1'], conv_param['stride'], conv_param['pad']) # 필터(가중치), 편향, 스트라이드, 패딩
    self.layers['Relu1'] = Relu()                                # 활성화 함수 ReLU
    self.layers['Pool1'] = Pooling(pool_h=2, pool_w=2, stride=2) # 풀링층, 2x2 Max pooling, 스트라이드 2
    
    # 2층 완전연결 계층
    self.layers['Affine1'] = Affine(self.params['W2'], self.params['b2'])
    self.layers['Relu2'] = Relu()                                # 활성화 함수 ReLU
    
    # 3층 완전연결 계층
    self.layers['Affine2'] = Affine(self.params['W3'], self.params['b3'])
    self.last_layer = SoftmaxWithLoss()                          # 출력층 Softmax-with-Loss 계층